# The Basics

## Authentication

You can authenticate to the device using a password or ssh keys. For automated scripts it is recommended to use SSH keys. For interactive scripts you can prompt the user for the parameters using the input() function and storing the results in a variable.

    hostname = input("Device Hostname:")
    username = input("Username:")
    password = input("Password:")
    
Since ` input() ` will show the password in cleartext you can use the getpass method:

    import getpass
    password = getpass.getpass()
    
The input from the example below is used in all subsequent sections.

*Hint: If you need to change these values just re-run this cell.*



In [ ]:
import getpass
hostname = input("Device Hostname:")
username = input("Username:")
password = getpass.getpass()

These variables can be used anywhere by putting calling them by name.

In [ ]:
print(hostname)
print(username)
print(password)

## Connecting to a device

It is possible to connect to a device via Serial, Telnet or SSH.

Before you can use PyEZ to connect to the device via SSH you must enable netconf with the following commands:

    set system services ssh
    set system services netconf ssh

Next you can initiate the connection to the device. Note that when SSH is used PyEZ automatically retrieves the device facts. Edit the code below:

In [ ]:
from jnpr.junos import Device
from pprint import pprint

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

pprint (dev.facts)

dev.close()

## Sample Output

    {'2RE': False,
     'HOME': '/cf/root',
     'RE0': {'last_reboot_reason': '0x10:misc hardware reason ',
             'model': 'VSRX RE',
             'status': 'Testing',
             'up_time': '23 hours, 36 minutes, 29 seconds'},
     'domain': None,
     'fqdn': 'vsrx2',
     'hostname': 'vsrx2',
     'ifd_style': 'CLASSIC',
     'model': 'VSRX',
     'personality': 'UNKNOWN',
     'serialnumber': '036aa3338cf5',
     'srx_cluster': False,
     'switch_style': 'NONE',
     'vc_capable': False,
     'version': '15.1X49-D50.3',
     'version_info': junos.version_info(major=(15, 1), type=X, minor=(49, 'D', 50), build=3)}